In [1]:
import pandas as pd
import tensorflow_addons as tfa
import tensorflow_core as tf
import tensorflow_core.keras as keras

In [2]:
tf.__version__

'2.1.0'

# Loading data

Read raw data

Source: http://www.macs.hw.ac.uk/InteractionLab/E2E/#data

In [3]:
raw_data = 'dataset/e2e/trainset.csv'
raw_df = pd.read_csv(raw_data)[['ref']]
raw_df.to_csv('dataset/e2e/trainset_prepared.csv', index=False, header=None)

Read prepared data

In [4]:
data = 'dataset/e2e/trainset_prepared.csv'
lines = tf.data.TextLineDataset(data)

for line in lines.take(3):
    print(line)

tf.Tensor(b'The Vaults pub near Caf\xc3\xa9 Adriatic has a 5 star rating.  Prices start at \xc2\xa330.', shape=(), dtype=string)
tf.Tensor(b'"Close to Caf\xc3\xa9 Brazil, The Cambridge Blue pub serves delicious Tuscan Beef for the cheap price of \xc2\xa310.50. Delicious Pub food."', shape=(), dtype=string)
tf.Tensor(b'The Eagle is a low rated coffee shop near Burger King and the riverside that is family friendly and is less than \xc2\xa320 for Japanese food.', shape=(), dtype=string)


# Preprocessing data

Split sentences into words

In [5]:
words = lines.map(tf.strings.split)
wordsets = words.unbatch().batch(11)

for row in wordsets.take(3):
    print(row)

tf.Tensor(
[b'The' b'Vaults' b'pub' b'near' b'Caf\xc3\xa9' b'Adriatic' b'has' b'a'
 b'5' b'star' b'rating.'], shape=(11,), dtype=string)
tf.Tensor(
[b'Prices' b'start' b'at' b'\xc2\xa330.' b'"Close' b'to' b'Caf\xc3\xa9'
 b'Brazil,' b'The' b'Cambridge' b'Blue'], shape=(11,), dtype=string)
tf.Tensor(
[b'pub' b'serves' b'delicious' b'Tuscan' b'Beef' b'for' b'the' b'cheap'
 b'price' b'of' b'\xc2\xa310.50.'], shape=(11,), dtype=string)


Get last words from sentences as label

In [6]:
def get_example_label(row):
    example = tf.strings.reduce_join(row[:-1], separator=' ')
    example = tf.expand_dims(example, axis=0)
    label = row[-1]
    return example, label

In [7]:
data = wordsets.map(get_example_label)
data = data.shuffle(1000)

In [8]:
for row in data.take(3):
    print(row)

(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'coffee shop that has Italian food and high customer rating.'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'It'>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'less than \xc2\xa320 price range with a low customer Rating.'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'It'>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'has a high customer rating, and has a price range'], dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'between'>)


Vectorize words

In [9]:
vocab_size = 5000

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size, output_sequence_length=10)

In [10]:
vectorize_layer.adapt(lines.batch(64))

Get vocabulary

In [11]:
vectorize_layer.get_vocabulary()[:5]

[b'the', b'is', b'a', b'food', b'in']

In [12]:
vectorize_layer.get_vocabulary()[-5:]

[b'1out5', b'19', b'130', b'110', b'0f']

# Creating model

In [16]:
class EncoderDecoder(tf.keras.models.Model):
    def __init__(self, max_features=5000, embedding_dim=100, rnn_units=32):
        super().__init__()
        self.max_features = max_features
        
        self.vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
            max_tokens=self.max_features, output_sequence_length=10)
        
        self.encoder_embedding = tf.keras.layers.Embedding(
            input_dim=self.max_features+1, output_dim=embedding_dim)
        self.lstm_layer = tf.keras.layers.LSTM(units=rnn_units)
        
        self.decoder_embedding = tf.keras.layers.Embedding(
            input_dim=self.max_features+1, output_dim=embedding_dim)
        
        projection_layer = tf.keras.layers.Dense(
            units=max_features)
        sampler = tfa.seq2seq.Sampler()
        
        self.decoder = tfa.seq2seq.BasicDecoder(
            cell=self.lstm_layer, sampler=sampler, output_layer=projection_layer)
        
        self.attention = tf.keras.layers.Attention()
        
    def train_step(self, data):
        X, y = data[0], data[1]
        x = self.vectorize_layer(x)
        y = self.vectorize_layer(y)[:, 0:1]
        y = tf.one_hot(y, depth=self.max_features)
        
        with tf.GradientTape() as tape:
            inputs = self.encoder_embedding(x)
            encoder_outputs, state_h, state_c = self.lstm_layer(inputs)
            
            attention_output = self.attention(encoder_outputs, state_h)
            attention_output = tf.expand_dims(attention_output, axis=1)
            
            targets = self.decoder_embedding(tf.zeros_like(y))
            concat_outputs = tf.concat([targets, attention_output], axis=-1)
            
            outputs, _, _ = self.decoder(concat_outputs, initial_state=[state_h, state_c])
            
#             gradients = tape.gradient(loss, )
    
    